# Create tweaked indexed aperture from file

We need an indexed aperture to be able to use the `SegmentedTelescope` from the `pastis` package. Creating a completely new one with `hcipy` doesn't work because the exact pixel-sampling won't work with the DM surfaces of the DM assisted vectro coronagraph (DMVC) anymore.

This is why in this notebook, I turn to getting and tweaking an indexed aperture from the input aperture file.

In [ ]:
import os
from astropy.io import fits
import hcipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import label

from pastis.config import CONFIG_PASTIS
import pastis.util

In [ ]:
matplotlib.rcParams['image.origin']='lower'

In [ ]:
# Load aperture file
datadir = os.path.join(pastis.util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR-B', 'optics_path_in_repo'))
aperture = fits.getdata(os.path.join(datadir, 'Pupil1.fits'))

plt.figure(figsize=(8,8))
plt.imshow(aperture, cmap='Greys_r')
plt.title('Input aperture file')

## Labelling the segments

Note how the results labels the segments bottom-up, and not in the circular fashion we know from real instruments. Also, the first segment starts with 1, while the non-segment background is 0, as are the segment gaps.  
Maybe I will get around to change the numbering at a later point.

In [ ]:
label_param = 0.154
segments, nseg = label(aperture > label_param)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(segments, cmap='Greys_r')
plt.title('Indexed aperture file')

In [ ]:
fits.writeto('aperture_indexed.fits', segments, overwrite=True)

## Calculate segment centers

In [ ]:
# Create pupil grid that is adjusted for the random input aperture we have
nPup = CONFIG_PASTIS.getfloat('LUVOIR-B', 'pupil_pixels')
nPup_arrays = aperture.shape[0]
D_pup = CONFIG_PASTIS.getfloat('LUVOIR-B', 'D_pup')

pupil_grid_arrays = hcipy.make_pupil_grid(nPup * (nPup_arrays / nPup), D_pup * (nPup_arrays / nPup))

In [ ]:
segment_positions = []
for i in range(nseg):
    seg = (segments == (i+1)).ravel()
    c_x = np.sum(seg * pupil_grid_arrays.x) / np.sum(seg)
    c_y = np.sum(seg * pupil_grid_arrays.y) / np.sum(seg)
    segment_positions.append((c_x, c_y))

In [ ]:
len(segment_positions)

## Create header with segment positions and save with indexed aperture to fits

In [ ]:
header_full = fits.Header()

In [ ]:
for i, pair in zip(range(nseg), segment_positions):
    header_full.set(f'SEG{i}_X', pair[0], f'x-position segment {i+1}')
    header_full.set(f'SEG{i}_Y', pair[1], f'y-position segment {i+1}') 

In [ ]:
# Write indexed aperture to file, with header containing segment coordinates
fits.writeto('aperture_LUVOIR-B_indexed.fits', segments, header_full, overwrite=True)

### Double-check segment positions

In [ ]:
aperture_field = hcipy.Field(np.reshape(aperture, nPup_arrays ** 2), pupil_grid_arrays)

In [ ]:
plt.figure(figsize=(8,8))
hcipy.imshow_field(aperture_field, cmap='Greys_r')
for i, pair in zip(range(nseg), segment_positions):
    plt.plot(pair[0], pair[1], marker='o')
plt.title('Indexed aperture file')